Copyright (c) 2018-2019 SumUp Analytics, Inc. All Rights Reserved.

NOTICE: All information contained herein is, and remains the property of SumUp Analytics Inc. and its suppliers, if any. The intellectual and technical concepts contained herein are proprietary to SumUp Analytics Inc. and its suppliers and may be covered by U.S. and Foreign Patents, patents in process, and are protected by trade secret or copyright law.

Dissemination of this information or reproduction of this material is strictly forbidden unless prior written permission is obtained from SumUp Analytics Inc.

# Initialization, configure API host and key, and create new API instance

In [1]:
import os
import csv
import json
import datetime
import time
import nucleus_api
from nucleus_api.rest import ApiException
import nucleus_api.api.nucleus_api as nucleus_helper
from pprint import pprint
import numpy as np
from pathlib import Path

# Determine if in Jupyter notebook or not
try:
    ip = get_ipython()
    running_notebook = True
except NameError:
    running_notebook = False

if running_notebook:
    print('Running example in Jupyter Notebook')
else:
    print('Running example in script mode')
    
configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

configuration.host = 'http://localhost:5000'
configuration.api_key['x-api-key'] = 'FsdP1JjrUIw5pui2lhWlqg'


# Create API instance
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

Running example in Jupyter Notebook


# Dataset APIs

## Append file from local drive to dataset

In [2]:
print('--------- Append file from local drive to dataset -----------')
dataset = "dataset_test"
file = 'quarles20181109a.pdf'         # file | 
metadata = {"time": "1/2/2018", 
            "author": "Test Author"}  # Optional json containing additional document metadata

try:
    api_response = api_instance.post_upload_file(file, dataset, metadata=metadata)
    fp = api_response.result
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset,)
    #print('api_response=', api_response)   # raw API response    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

print('-------------------------------------------------------------')

--------- Append file from local drive to dataset -----------
quarles20181109a.pdf ( 51187 bytes) has been added to dataset dataset_test
-------------------------------------------------------------


# Append all PDFs from a folder to dataset in parallel

In [6]:
folder = 'fomc-minutes'         
dataset = 'dataset_test'# str | Destination dataset where the file will be inserted.
print('--------- Append all files from local folder {} to dataset {} in parallel -----------'.format(folder, dataset))

# build file iterable. Each item in the iterable is in the format below:
# {'filename': filename,   # filename to be uploaded. REQUIRED
#  'metadata': {           # metadata for the file. Optional
#      'key1': val1,       # keys can have arbiturary names as long as the names only
#      'key2': val2        # contain alphanumeric (0-9|a-z|A-Z) and underscore (_)
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        if Path(file).suffix == '.pdf':
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'field1': 'financial'}}
            file_iter.append(file_dict)


file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=1)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)
    
print('-------------------------------------------------------------')

--------- Append all files from local folder fomc-minutes to dataset dataset_test in parallel -----------
fomcminutes20181219.pdf ( 495324 bytes) has been added to dataset dataset_test
fomcminutes20181108.pdf ( 369090 bytes) has been added to dataset dataset_test
-------------------------------------------------------------


## Append file from URL to dataset

In [7]:
print('------------ Append file from URL to dataset ---------------')

dataset = 'dataset_test'
file_url = 'https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109a.docx'
# Optional filename saved on the server for the URL. If not specified, Nucleus will make
# an intelligent guess from the file URL
filename = 'quarles20181109a-newname.pdf'  
payload = nucleus_api.UploadURLModel(
                dataset=dataset,
                file_url=file_url,
                filename=filename  
            ) # UploadURLModel | 

try:
    api_response = api_instance.post_upload_url(payload)
    #print('api_response=', api_response)   # raw API response
    url_prop = api_response.result
    print(url_prop.file_url, '(', url_prop.size, ' bytes) has been added to dataset', dataset)

except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    

print('-------------------------------------------------------------')

------------ Append file from URL to dataset ---------------
https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109a.docx ( 16881  bytes) has been added to dataset dataset_test
-------------------------------------------------------------


## Append files from URLs to dataset in parallel

In [8]:
print('------------ Append file from URL to dataset ---------------')

dataset = 'dataset_test'
file_urls = [
    'https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109a.docx',
    'https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109b.docx',
    'https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109c.docx',
    'https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109d.docx'
]

url_props = nucleus_helper.upload_urls(api_instance, dataset, file_urls, processes=1)

for up in url_props:
    print(up.file_url, '(', up.size, ' bytes) has been added to dataset', dataset)
    

print('-------------------------------------------------------------')

------------ Append file from URL to dataset ---------------
https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109a.docx ( 16881  bytes) has been added to dataset dataset_test
https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109b.docx ( 16881  bytes) has been added to dataset dataset_test
https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109c.docx ( 16881  bytes) has been added to dataset dataset_test
https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109d.docx ( 16881  bytes) has been added to dataset dataset_test
-------------------------------------------------------------


## Append JSON to dataset

In [9]:
dataset = 'dataset_test'
print('----------- Append json from to dataset {}-----------------'.format(dataset))

document = {
    "title": "This a test json title field",
    "time": "2019-01-01",
    "content": "This is a test json content field"
}
document = {'time': datetime.datetime(2019, 4, 5, 22, 4, 31, 417723), 
            'title': '0', 
            'content': ['if only mitt romney could turn the olympic torch on the newspaper headlines in london hes the party pooper in the daily mail nowhere man in the times of london and mitt the twit in the sun this was not the storyline romney and his team wanted when they journeyed overseas for a trip designed to burnish the gop contenders foreign policy credentials romney has yet to publicly acknowledge the outrage he set off in london when he appeared to question the citys disconcerting problems in gearing up for the olympic games in an interview with cnns piers morgan on thursday romney chuckled when he was asked about the criticism romneys olympics false start well im delighted to see the kind of support that has been around the torch for instance i watched last night on bbc an entire program about the torch being run across great britain and the kind of crowds i guess millions of people that turned out to see the torch thats what you hope to see romney told morgan asked about the controversy on nbcs today show romney again sidestepped the question but declared london prepared after being here a couple of days it looks to me like london is ready romney said the uproar in britain reached its crescendo thursday evening when london mayor boris johnson whipped up a crowd of revelers at a preolympics celebration with a taunt for the republican presidential candidate johnson is a tory theoretically putting him near romney on the same conservative end of the political spectrum theres this guy called mitt romney who wants to know if we are ready are we ready yes we are johnson shouted at what sounded like a political rally for president obama meanwhile on twitter the hashtag romneyshambles was trending on both sides of the atlantic for instance'], 'author': 'CNN'}
payload = nucleus_api.Appendjsonparams(dataset=dataset,
                                       document=document
                                      )

try:
    api_response = api_instance.post_append_json_to_dataset(payload)
    print(api_response.result)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

----------- Append json from to dataset dataset_test-----------------
{'n_documents': 10, 'size': 1750}


## Append jsons from csv to dataset in parallel

In [14]:
# This dataset will be used to test all topics and documents APIs
csv_file = 'trump-tweets-100.csv'
dataset = 'trump_tweets1'
print('----------- Append json from CSV {} to dataset {}-----------------'.format(csv_file, dataset))

with open(csv_file, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    json_props = nucleus_helper.upload_jsons(api_instance, dataset, reader, processes=1)
    
    total_size = 0
    total_jsons = 0
    for jp in json_props:
        total_size += jp.size
        total_jsons += 1
        
    print(total_jsons, 'JSON records (', total_size, 'bytes) appended to', dataset)

print('-------------------------------------------------------------')

----------- Append json from CSV trump-tweets-100.csv to dataset trump_tweets1-----------------
100 JSON records ( 20885 bytes) appended to trump_tweets1
-------------------------------------------------------------


## List available datasets

In [11]:
print('---------------- List available datasets ---------------------')
try:
    api_response = api_instance.get_list_datasets()
except ApiException as e:
    print("Exception when calling DatasetsApi->get_list_datasets: %s\n" % e)

list_datasets = api_response.result

print(len(list_datasets), 'datasets in the database:')
for ds in list_datasets:
    print('    ', ds)
    
print('-------------------------------------------------------------')

---------------- List available datasets ---------------------
2 datasets in the database:
     dataset_test
     trump_tweets
-------------------------------------------------------------


## Get dataset information

In [12]:
dataset = 'dataset_test' # str | Dataset name.
print('--------------- Get dataset information from {}-------------------'.format(dataset))

query = '' # str | Fulltext query, using mysql MATCH boolean query format. (optional)
metadata_selection = '' # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)
time_period = '' # str | Time period selection (optional)
period_start = "" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"

try:
    payload = nucleus_api.DatasetInfo(dataset=dataset, 
                                    query=query, 
                                    metadata_selection=metadata_selection, 
                                    time_period=time_period)
    api_response = api_instance.post_dataset_info(payload)
    print('Information about dataset', dataset)
    print('    Language:', api_response.result.detected_language)
    print('    Number of documents:', api_response.result.num_documents)
    print('    Time range:', datetime.datetime.fromtimestamp(float(api_response.result.time_range[0])),
             'to', datetime.datetime.fromtimestamp(float(api_response.result.time_range[1])))
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])


print('-------------------------------------------------------------')

--------------- Get dataset information from dataset_test-------------------
Information about dataset dataset_test
    Language: en
    Number of documents: 10
    Time range: 2018-01-02 00:00:00 to 2019-04-07 04:39:27
-------------------------------------------------------------


## Delete document

In [13]:
#print('--------------------- Delete document -----------------------')
#dataset = 'dataset_test'

#docid = '1'
#payload = nucleus_api.Deletedocumentmodel(dataset=dataset,
#                                          docid=docid) # Deletedocumentmodel | 

#try:
#    api_response = api_instance.post_delete_document(payload)
#except ApiException as e:
#    print("Exception when calling DatasetsApi->post_delete_document: %s\n" % e)


#print('Document', docid, 'from dataset', dataset, 'has been deleted.')
## print(api_response)     # raw API response
#print('-------------------------------------------------------------')

## Delete dataset

In [14]:
print('--------------------- Delete dataset ------------------------')

dataset = 'dataset_test'
payload = nucleus_api.Deletedatasetmodel(dataset=dataset) # Deletedatasetmodel | 


try:
    api_response = api_instance.post_delete_dataset(payload)
    print(api_response.result['result'])
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    
# List datasets again to check if the specified dataset has been deleted
try:
    api_response = api_instance.get_list_datasets()
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    
print('-------------------------------------------------------------')

--------------------- Delete dataset ------------------------
Dataset dataset_test has been deleted.
-------------------------------------------------------------


# Topic APIs

## Get list of topics from dataset

In [15]:
dataset = 'trump_tweets'
print('------------- Get list of topics from dataset {}--------------'.format(dataset))

#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)

try:
    payload = nucleus_api.Topics(dataset=dataset,                                
                                query=query,                   
                                custom_stop_words=custom_stop_words,     
                                num_topics=num_topics,
                                metadata_selection=metadata_selection,
                                time_period=time_period)
    api_response = api_instance.post_topic_api(payload)        
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

doc_ids = api_response.result.doc_ids
topics = api_response.result.topics
for i, res in enumerate(topics):
    print('Topic', i, 'keywords:')
    print('    Keywords:', res.keywords)
    keywords_weight_str = ";".join(str(x) for x in res.keywords_weight)
    print('    Keyword weights:', keywords_weight_str)
    print('    Strength:', res.strength)
    doc_topic_exposure_sel = []  # list of non-zero doc_topic_exposures
    doc_id_sel = []        # list of doc ids matching doc_topic_exposure_sel
    for j in range(len(res.doc_topic_exposures)):
        doc_topic_exp = float(res.doc_topic_exposures[j])
        if doc_topic_exp != 0:
            doc_topic_exposure_sel.append(doc_topic_exp)
            doc_id_sel.append(doc_ids[j])
    
    doc_id_sel_str = ' '.join(str(x) for x in doc_id_sel)
    doc_topic_exposure_sel_str = ' '.join(str(x) for x in doc_topic_exposure_sel)
    print('    Document IDs:', doc_id_sel_str)
    print('    Document exposures:', doc_topic_exposure_sel_str)

    print('---------------')
    
print('-------------------------------------------------------------')

------------- Get list of topics from dataset trump_tweets--------------
Topic 0 keywords:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;entered legally
    Keyword weights: 0.1241;0.1241;0.1241;0.2483;0.1241;0.1241;0.0655;0.0655
    Strength: 0.1274
    Document IDs: 3397215194896514820 4825367511331474696
    Document exposures: 0.527691 0.472309
---------------
Topic 1 keywords:
    Keywords: bruce ohr;ohr justice;justice department;christopher steele;helping disgraced;disgraced christopher;believe accused;accused helping
    Keyword weights: 0.2587;0.1342;0.1342;0.0879;0.0879;0.0879;0.0879;0.1213
    Strength: 0.1584
    Document IDs: 6303783743713708484 7290029718334628379 12936417737022695482 13072902166717108911 13460407141547160473 16828254283062838304
    Document exposures: 0.306009 0.127398 0.162947 0.089779 0.13714 0.176727
---------------
Topic 2 keywords:
    Keywords: america great;statement a

## Get list of topics from dataset with a time range selection

In [16]:
dataset = 'trump_tweets'
print('------------- Get list of topics from dataset {}--------------'.format(dataset))

#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
period_start = "2016-10-15" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD"
period_end = "2019-01-01" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD"

try:
    payload = nucleus_api.Topics(dataset=dataset,                                
                                query=query,                   
                                custom_stop_words=custom_stop_words,     
                                num_topics=num_topics,
                                metadata_selection=metadata_selection,
                                period_start=period_start,
                                period_end=period_end)
    api_response = api_instance.post_topic_api(payload)        
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    
doc_ids = api_response.result.doc_ids
topics = api_response.result.topics
for i, res in enumerate(topics):
    print('Topic', i, 'keywords:')
    print('    Keywords:', res.keywords)
    keywords_weight_str = ";".join(str(x) for x in res.keywords_weight)
    print('    Keyword weights:', keywords_weight_str)
    print('    Strength:', res.strength)
    doc_topic_exposure_sel = []  # list of non-zero doc_topic_exposure
    doc_id_sel = []        # list of doc ids matching doc_topic_exposure_sel
    for j in range(len(res.doc_topic_exposures)):
        doc_topic_exp = float(res.doc_topic_exposures[j])
        if doc_topic_exp != 0:
            doc_topic_exposure_sel.append(doc_topic_exp)
            doc_id_sel.append(doc_ids[j])
    
    doc_id_sel_str = ' '.join(str(x) for x in doc_id_sel)
    doc_topic_exposure_sel_str = ' '.join(str(x) for x in doc_topic_exposure_sel)
    print('    Document IDs:', doc_id_sel_str)
    print('    Document exposures:', doc_topic_exposure_sel_str)

    print('---------------')

print('-------------------------------------------------------------')

------------- Get list of topics from dataset trump_tweets--------------
Topic 0 keywords:
    Keywords: total endorsement;complete total;pete complete;bob total;america great;witch hunt;great great;rigged witch
    Keyword weights: 0.4175;0.4831;0.0008;0.0005;0.0004;0.0006;0.0339;0.0632
    Strength: 0.2965
    Document IDs: 1292265014981711161 2205902445999073018 2373450842905457495 3545423942726121399 5217366909427623007 5864841412738683134 8047817457772465264 8073561612845847316 8991483632660067955 9785400758777816854 10010199882756041615 10785765889843731879 11760987759040078706 12936417737022695482 12945860735388879748 14722230792170818214 14988735547592816033 15429790537822270921 16259624839192846495 16485451943305749356 16828254283062838304 17770243791305455215 18423518517048905190
    Document exposures: 0.119382 0.000107 0.108337 0.000188 0.000142 0.000196 9.1e-05 0.134681 0.061866 0.119382 0.000217 0.119382 0.115334 0.000109 9.1e-05 9.1e-05 7.3e-05 7.6e-05 0.060438 0.159343 

## Get list of topics from dataset with a metadata selection

In [17]:
dataset = 'trump_tweets'
print('------------- Get list of topics from dataset {}--------------'.format(dataset))

#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
metadata_selection = {"author": "D_Trump16"} # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)

try:
    payload = nucleus_api.Topics(dataset=dataset,                                
                                query=query,                   
                                custom_stop_words=custom_stop_words,     
                                num_topics=num_topics,
                                metadata_selection=metadata_selection)
    api_response = api_instance.post_topic_api(payload)        
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    
#print(api_response)
doc_ids = api_response.result.doc_ids
topics = api_response.result.topics
for i, res in enumerate(topics):
    print('Topic', i, 'keywords:')
    print('    Keywords:', res.keywords)
    keywords_weight_str = ";".join(str(x) for x in res.keywords_weight)
    print('    Keyword weights:', keywords_weight_str)
    print('    Strength:', res.strength)
    doc_topic_exposure_sel = []  # list of non-zero doc_topic_exposure
    doc_id_sel = []        # list of doc ids matching doc_topic_exposure_sel
    for j in range(len(res.doc_topic_exposures)):
        doc_topic_exp = float(res.doc_topic_exposures[j])
        if doc_topic_exp != 0:
            doc_topic_exposure_sel.append(doc_topic_exp)
            doc_id_sel.append(doc_ids[j])
    
    doc_id_sel_str = ' '.join(str(x) for x in doc_id_sel)
    doc_topic_exposure_sel_str = ' '.join(str(x) for x in doc_topic_exposure_sel)
    print('    Document IDs:', doc_id_sel_str)
    print('    Document exposures:', doc_topic_exposure_sel_str)

    print('---------------')
    
print('-------------------------------------------------------------')

------------- Get list of topics from dataset trump_tweets--------------
Topic 0 keywords:
    Keywords: worse hightax;hightax andrew;andrew cuomo
    Keyword weights: 0.3333;0.3333;0.3333
    Strength: 0.3333
    Document IDs: 5864841412738683134
    Document exposures: 1.0
---------------
Topic 1 keywords:
    Keywords: statement america;cuomo statement;america great
    Keyword weights: 0.3333;0.3333;0.3333
    Strength: 0.3333
    Document IDs: 5864841412738683134
    Document exposures: 1.0
---------------
Topic 2 keywords:
    Keywords: hillary clinton;great hillary;great great
    Keyword weights: 0.3333;0.3333;0.3333
    Strength: 0.3333
    Document IDs: 5864841412738683134
    Document exposures: 1.0
---------------
-------------------------------------------------------------


## Get topic summary

In [2]:
dataset = 'trump_tweets'
print('------------------- Get topic summary for {} -----------------------'.format(dataset))
 # str | Dataset name.
#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
summary_length = 6 # int | The maximum number of bullet points a user wants to see in each topic summary. (optional) (default to 6)
context_amount = 0 # int | The number of sentences surrounding key summary sentences in the documents that they come from. (optional) (default to 0)
num_docs = 20 # int | The maximum number of key documents to use for summarization. (optional) (default to 20)
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""]  (optional)
period_start = "" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
api_response = None

try:
    payload = nucleus_api.TopicSummaryModel	(
        dataset=dataset, 
        query=query,
        custom_stop_words=custom_stop_words, 
        num_topics=num_topics, 
        num_keywords=num_keywords,
        metadata_selection=metadata_selection,
        summary_length=summary_length, 
        context_amount=context_amount, 
        num_docs=num_docs)
    api_response = api_instance.post_topic_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    for i,res in enumerate(api_response.result):
        print('Topic', i, 'summary:')
        print('    Keywords:', res.keywords)
        for j in range(len(res.summary)):
            print(res.summary[j])
            print('    Document ID:', res.summary[j].sourceid)
            print('        Title:', res.summary[j].title)
            print('        Sentences:', res.summary[j].sentences)
            print('        Author:', res.summary[j].attribute['author'])
            print('        Time:', datetime.datetime.fromtimestamp(float(res.summary[j].attribute['time'])))

        print('---------------')
    
print('-------------------------------------------------------------')

------------------- Get topic summary for trump_tweets -----------------------
Topic 0 summary:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;entered legally
{'attribute': {'author': 'D_Trump57', 'counts': None, 'time': 1534339080},
 'sentences': "['“People who enter the United States without our permission "
              'are illegal aliens and illegal aliens should not be treated the '
              "same as people who entered the US legally”.']",
 'sourceid': '3397215194896514820',
 'title': 'D_Trump2018_8_15_13_18'}
    Document ID: 3397215194896514820
        Title: D_Trump2018_8_15_13_18
        Sentences: ['“People who enter the United States without our permission are illegal aliens and illegal aliens should not be treated the same as people who entered the US legally”.']
        Author: D_Trump57
        Time: 2018-08-15 06:18:00
{'attribute': {'author': 'D_Trump63', 'counts': None, 'time': 1534337

## Get topic sentiment

In [3]:
dataset = 'trump_tweets' # str | Dataset name
print('---------------- Get topic sentiment for {} ------------------------'.format(dataset))

#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
custom_dict_file = {"great": 1.0, "awful": -1.0, "clinton":-1.0, "trump":1.0} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
period_start = "" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"

try:
    payload = nucleus_api.TopicSentimentModel(
        dataset=dataset, 
        query=query, 
        custom_stop_words=custom_stop_words, 
        num_topics=num_topics, 
        num_keywords=num_keywords,
        custom_dict_file=custom_dict_file)
    api_response = api_instance.post_topic_sentiment_api(payload)
    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

print(api_response)
for i,res in enumerate(api_response.result):
    print('Topic', i, 'sentiment:')
    print('    Keywords:', res.keywords)
    print('    Sentiment:', res.sentiment)
    print('    Strength:', res.strength)
    
    doc_id_str = ' '.join(str(x) for x in res.doc_ids)
    doc_sentiment_str = ' '.join(str(x) for x in res.doc_sentiments)
    doc_score_str = ' '.join(str(x) for x in res.doc_topic_exposures)
    print('    Document IDs:', doc_id_str)
    print('    Document Sentiments:', doc_sentiment_str)
    print('    Document Exposures:', doc_score_str)
    
    print('---------------')
    
print('-------------------------------------------------------------')

---------------- Get topic sentiment for trump_tweets ------------------------
{'job_id': '56380',
 'result': [{'doc_ids': ['3397215194896514820', '4825367511331474696'],
             'doc_sentiments': ['0.0', '0.0'],
             'doc_topic_exposures': ['[0.527691, 0.472309]'],
             'keywords': 'illegal aliens;united permission;permission '
                         'illegal;enter united;aliens treated;aliens '
                         'illegal;treated entered;entered legally',
             'sentiment': '0.0',
             'strength': '0.1274'},
            {'doc_ids': ['6303783743713708484',
                         '7290029718334628379',
                         '12936417737022695482',
                         '13072902166717108911',
                         '13460407141547160473',
                         '16828254283062838304'],
             'doc_sentiments': ['0.090909',
                                '0.0',
                                '0.133333',
                    

## Get topic consensus

In [5]:
dataset = 'trump_tweets' # str | Dataset name.
print('---------------- Get topic consensus for {} ------------------------'.format(dataset))

query = '' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
excluded_docs = [''] # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
custom_dict_file = {"great": 1.0, "awful": -1.0, "clinton":-1.0, "trump":1.0} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = ""     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
period_start = "" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"

try:
    payload = nucleus_api.TopicConsensusModel(
        dataset=dataset, 
        query=query, 
        custom_stop_words=custom_stop_words, 
        num_topics=num_topics, 
        num_keywords=num_keywords,
        custom_dict_file=custom_dict_file)
    api_response = api_instance.post_topic_consensus_api(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    
for i, res in enumerate(api_response.result):
    print('Topic', i, 'consensus:')
    print('    Keywords:', res.keywords)
    print('    Consensus:', res.consensus)
    print('    Strength:', res.strength)
    
    print('---------------')
    
print('-------------------------------------------------------------')

---------------- Get topic consensus for trump_tweets ------------------------
Topic 0 consensus:
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;entered legally
    Consensus: 1.0
    Strength: 0.1274
---------------
Topic 1 consensus:
    Keywords: bruce ohr;ohr justice;justice department;christopher steele;helping disgraced;disgraced christopher;believe accused;accused helping
    Consensus: 0.5271
    Strength: 0.1584
---------------
Topic 2 consensus:
    Keywords: america great;statement america;andrew cuomo;great great;hillary clinton;worse hightax;hightax andrew;cuomo statement
    Consensus: 0.8127
    Strength: 0.1414
---------------
Topic 3 consensus:
    Keywords: witch hunt;donald trump;rigged witch;frame donald;unfortunate situation;trump rigged;situation decided;decided frame
    Consensus: 0.5724
    Strength: 0.1446
---------------
Topic 4 consensus:
    Keywords: peter strzok;fired fbi;agent 

## Get topic historical analysis

In [6]:
dataset = 'trump_tweets'   # str | Dataset name.
print('------------ Get topic historical analysis for {} ----------------'.format(dataset))

update_period = 'm' # str | Frequency at which the historical anlaysis is performed. choices=["d","m","H","M"] (default to d)
query = '' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
custom_stop_words = ["real","hillary"] # str | List of stop words (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
inc_step = 1 # int | Number of increments of the udpate period in between two historical computations. (optional) (default to 1)
excluded_docs = [''] # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
custom_dict_file = {} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = "12M"     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
period_start = "" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
api_response = None
try:
    payload = nucleus_api.TopicHistoryModel(
        dataset=dataset, 
        time_period=time_period, 
        update_period=update_period, 
        query=query, 
        custom_stop_words=custom_stop_words, 
        num_topics=num_topics, 
        num_keywords=num_keywords, 
        metadata_selection=metadata_selection, 
        inc_step=inc_step, 
        excluded_docs=excluded_docs,
        custom_dict_file=custom_dict_file)
    api_response = api_instance.post_topic_historical_analysis_api(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    print(e)

print('Printing historical metrics data...')
print('NOTE: historical metrics data can be plotted when running the example in Jupyter Notebook')

for i,res in enumerate(api_response.result):
    print('Topic', i, res.keywords)
    print('    Timestamps:', res.time_stamps)
    print('    Strengths:', res.strengths)
    print('    Consensuses:', res.consensuses)
    print('    Sentiments:', res.sentiments)
    print('----------------')
            

# chart the historical metrics when running in Jupyter Notebook
if running_notebook:
    print('Plotting historical metrics data...')
    historical_metrics = []
    for res in api_response.result:
        # conctruct a list of historical metrics dictionaries for charting
        historical_metrics.append({
            'topic'    : res.keywords,
            'time_stamps' : np.array(res.time_stamps),
            'strength' : np.array(res.strengths, dtype=np.float32),
            'consensus': np.array(res.consensuses, dtype=np.float32), 
            'sentiment': np.array(res.sentiments, dtype=np.float32)})

    selected_topics = range(len(historical_metrics)) 
    #nucleus_helper.topic_charts_historical(historical_metrics, selected_topics, True)

print('-------------------------------------------------------------')

------------ Get topic historical analysis for trump_tweets ----------------


KeyboardInterrupt: 

## Get author connectivity

In [12]:
dataset = 'trump_tweets' # str | Dataset name.
print('----------------- Get author connectivity for {} -------------------'.format(dataset))

target_author = 'D_Trump16' # str | Name of the author to be analyzed.
query = '' # str | Fulltext query, using mysql MATCH boolean query format. Subject covered by the author, on which to focus the analysis of connectivity. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
custom_stop_words = ["real","hillary"] # str | List of words possibly used by the target author that are considered not information-bearing. (optional)
excluded_docs = [''] # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)

metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
time_period = "12M"     # str | Time period selection. Choices: ["1M","3M","6M","12M","3Y","5Y",""] (optional)
period_start = "" # str | Start date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"
period_end = "" # str | End date for the period to analyze within the dataset. Format: "YYYY-MM-DD HH:MM:SS"

try:
    payload = nucleus_api.AuthorConnection(dataset=dataset, 
                                            target_author=target_author, 
                                            query=query, 
                                            custom_stop_words=custom_stop_words, 
                                            time_period=time_period, 
                                            metadata_selection=metadata_selection, 
                                            excluded_docs=excluded_docs)
    api_response = api_instance.post_author_connectivity_api(payload)    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

#print(api_response)
res = api_response.result
print('Mainstream connections:')
for mc in res.mainstream_connections:
    print('    Keywords:', mc.keywords)
    print('    Authors:', " ".join(str(x) for x in mc.authors))
    
print('Niche connections:')
for nc in res.niche_connections:
    print('    Keywords:', nc.keywords)
    print('    Authors:', " ".join(str(x) for x in nc.authors))  
    
#pprint(api_response)   # raw API response
print('-------------------------------------------------------------')

----------------- Get author connectivity for trump_tweets -------------------
{'job_id': '56389',
 'result': {'mainstream_connections': [{'authors': ['D_Trump44'],
                                        'keywords': 'worse hightax;hightax '
                                                    'andrew;andrew cuomo'}],
            'niche_connections': [{'authors': ['D_Trump24'],
                                   'keywords': 'statement america;cuomo '
                                               'statement;america great'}]}}
Mainstream connections:
    Keywords: worse hightax;hightax andrew;andrew cuomo
    Authors: D_Trump44
Niche connections:
    Keywords: statement america;cuomo statement;america great
    Authors: D_Trump24
-------------------------------------------------------------


## Get topic transfer

In [15]:
dataset0 = 'trump_tweets'
print('------------------- Get topic transfer for {} -----------------------'.format(dataset))

dataset1 = 'trump_tweets1' # str | Validation dataset (optional if period_0 and period_1 dates provided)
#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [""] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
period_0_start = '2018-08-12' # Not needed if you provide a validation dataset in the "dataset1" variable 
period_0_end = '2018-08-15' # Not needed if you provide a validation dataset in the "dataset1" variable
period_1_start = '2018-08-16' # Not needed if you provide a validation dataset in the "dataset1" variable
period_1_end = '2018-08-19' # Not needed if you provide a validation dataset in the "dataset1" variable
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)

try:
    payload = nucleus_api.TopicTransferModel(dataset0=dataset0, 
                                            query=query, 
                                            custom_stop_words=custom_stop_words, 
                                            num_topics=num_topics, 
                                            num_keywords=num_keywords,
                                            period_0_start=period_0_start,
                                            period_0_end=period_0_end,
                                            period_1_start=period_1_start,
                                            period_1_end=period_1_end,
                                            metadata_selection=metadata_selection)
    api_response = api_instance.post_topic_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    doc_ids_t1 = api_response.result.doc_ids_t1
    topics = api_response.result.topics
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Strength:', res.strength)
        print('    Document IDs:', doc_ids_t1)
        print('    Exposure per Doc in Validation Dataset:', res.doc_topic_exposures_t1)
        print('---------------')
    
print('-------------------------------------------------------------')

------------------- Get topic transfer for trump_tweets1 -----------------------
Topic 0 exposure within validation dataset:
    Keywords: trump campaign;lou dobbs;evidence collusion;dobbs special;democrats evidence;conflicts angry;collusion trump;angry democrats
    Strength: 0.1279
    Document IDs: ['950604085993420810', '1380411530707030282', '2205902445999073018', '2383865888350638791', '2952292854093486503', '3325720912382988533', '3328210989202773955', '3499421997204683102', '3545423942726121399', '3683627708016583172', '4625946039318940221', '5098711196746147249', '5217366909427623007', '5566900818722282521', '5620968974223273808', '5864841412738683134', '7014079786619530089', '7180359259391996839', '7242230233701612989', '7290029718334628379', '7967605045913198983', '8047817457772465264', '8192928964490616283', '9035906359710233744', '10006474250568936611', '10010199882756041615', '10370745183868017022', '11198341698462345569', '11357478787751126246', '11767302015801488535', '

## Get topic sentiment transfer

In [20]:
dataset0 = 'trump_tweets'
print('------------------- Get topic sentiment transfer for {} -----------------------'.format(dataset))

#dataset1 = dataset # str | Validation dataset (optional if period_0 and period_1 dates provided)
#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [""] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
period_0_start = '2018-08-12' # Not needed if you provide a validation dataset in the "dataset1" variable 
period_0_end = '2018-08-15' # Not needed if you provide a validation dataset in the "dataset1" variable
period_1_start = '2018-08-16' # Not needed if you provide a validation dataset in the "dataset1" variable
period_1_end = '2018-08-19' # Not needed if you provide a validation dataset in the "dataset1" variable
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
custom_dict_file = {"great": 1.0, "awful": -1.0, "clinton":-1.0, "trump":1.0} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)

try:
    payload = nucleus_api.TopicSentimentTransferModel(dataset0=dataset0, 
                                            query=query, 
                                            custom_stop_words=custom_stop_words, 
                                            num_topics=num_topics, 
                                            num_keywords=num_keywords,
                                            period_0_start=period_0_start,
                                            period_0_end=period_0_end,
                                            period_1_start=period_1_start,
                                            period_1_end=period_1_end,
                                            metadata_selection=metadata_selection,
                                            custom_dict_file=custom_dict_file)
    api_response = api_instance.post_topic_sentiment_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topics = api_response.result
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Strength:', res.strength)
        print('    Sentiment:', res.sentiment)
        print('    Document IDs:', res.doc_ids_t1)
        print('    Sentiment per Doc in Validation Dataset:', res.doc_sentiments_t1)
        print('---------------')
    
print('-------------------------------------------------------------')

------------------- Get topic sentiment transfer for trump_tweets1 -----------------------
ERROR: {'stack': 'Traceback (most recent call last):\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/job_manager.py", line 76, in subprocess_entrypoint\n    res = function(**expected_args)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/apis/topics.py", line 1196, in get_topic_sentiment_transfer\n    results_sent = top_sentiment.fit(dataset_t1, topics, model_t1)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/analytics/topic_sentiment.py", line 314, in fit\n    self.rows)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/analytics/topic_sentiment.py", line 49, in topic_sentiment\n    return serial_topic_sentiment(d, topics, model, custom_dict, sent_vec = None, docids= None, sentence_scores = None, rows = None)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/analytics/topic_sentiment.py", line 100, in serial_topic_sentiment\n    doc_word_counts = np.ad

## Get topic consensus transfer

In [24]:
dataset0 = 'trump_tweets'
print('------------------- Get topic consensus transfer for {} -----------------------'.format(dataset))

#dataset1 = dataset # str | Validation dataset (optional if period_0 and period_1 dates provided)
#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [""] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
metadata_selection = "" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
period_0_start = '2018-08-12' # Not needed if you provide a validation dataset in the "dataset1" variable 
period_0_end = '2018-08-15' # Not needed if you provide a validation dataset in the "dataset1" variable
period_1_start = '2018-08-16' # Not needed if you provide a validation dataset in the "dataset1" variable
period_1_end = '2018-08-19' # Not needed if you provide a validation dataset in the "dataset1" variable
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
custom_dict_file = {"great": 1.0, "awful": -1.0, "clinton":-1.0, "trump":1.0} # file | Custom sentiment dictionary JSON file. Example, {"field1": value1, ..., "fieldN": valueN} (optional)

try:
    payload = nucleus_api.TopicConsensusTransferModel(dataset0=dataset0, 
                                            query=query, 
                                            custom_stop_words=custom_stop_words, 
                                            num_topics=num_topics, 
                                            num_keywords=num_keywords,
                                            period_0_start=period_0_start,
                                            period_0_end=period_0_end,
                                            period_1_start=period_1_start,
                                            period_1_end=period_1_end,
                                            metadata_selection=metadata_selection,
                                            custom_dict_file=custom_dict_file)
    api_response = api_instance.post_topic_consensus_transfer_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topics = api_response.result
    for i,res in enumerate(topics):
        print('Topic', i, 'exposure within validation dataset:')
        print('    Keywords:', res.keywords)
        print('    Consensus:', res.consensus)
        print('---------------')
    
print('-------------------------------------------------------------')

------------------- Get topic consensus transfer for trump_tweets1 -----------------------
ERROR: {'stack': 'Traceback (most recent call last):\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/job_manager.py", line 76, in subprocess_entrypoint\n    res = function(**expected_args)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/rest_api/apis/topics.py", line 1318, in get_topic_consensus_transfer\n    results_cons = top_consensus.fit(dataset_t1, topics, model_t1)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/analytics/topic_herding.py", line 87, in fit\n    herding = topic_herding(d, topics, model, self.custom_dict, self.bundle)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/analytics/topic_herding.py", line 30, in topic_herding\n    ret = topic_sentiment(d, topics, model, custom_dict)\n  File "/Users/jimw/github/sumup/nucleus-api/nucleus/analytics/topic_sentiment.py", line 49, in topic_sentiment\n    return serial_topic_sentiment(d, topics, model, cust

## Get topic delta

In [25]:
dataset = 'trump_tweets'
print('------------------- Get topic deltas for {} -----------------------'.format(dataset))
 
#dataset = dataset # str | Dataset name.
#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = [""] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
metadata_selection ="" # dict | JSON object specifying metadata-based queries on the dataset, of type {"metadata_field": "selected_values"} (optional)
period_0_start = '2018-08-12'
period_0_end = '2018-08-15'
period_1_start = '2018-08-16'
period_1_end = '2018-08-19'
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)

try:
    payload = nucleus_api.TopicDeltaModel(
        dataset=dataset, 
        query=query, 
        custom_stop_words=custom_stop_words, 
        num_topics=num_topics, 
        num_keywords=num_keywords,
        period_0_start=period_0_start,
        period_0_end=period_0_end,
        period_1_start=period_1_start,
        period_1_end=period_1_end,
        metadata_selection=metadata_selection)
    api_response = api_instance.post_topic_delta_api(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

doc_ids_t0 = api_response.result.doc_ids_t0
doc_ids_t1 = api_response.result.doc_ids_t1
topics = api_response.result.topics
for i,res in enumerate(topics):
    print('Topic', i, 'changes in exposure:')
    print('    Keywords:', res.keywords)
    print('    Document ID:', doc_ids_t0, doc_ids_t1)
    print('    Per Source Change in Exposure:', res.doc_topic_exposure_deltas)
    print('---------------')
    
print('-------------------------------------------------------------')

------------------- Get topic deltas for trump_tweets -----------------------
Topic 0 changes in exposure:
    Keywords: trump campaign;lou dobbs;evidence collusion;dobbs special;democrats evidence;conflicts angry;collusion trump;angry democrats
    Document ID: ['372746459070796601', '776902852041351634', '1620156333107313580', '1854520462215508183', '2365960778917245307', '4555868983588618437', '4746121785136787662', '4767189974744133712', '5821020073909755150', '6173618630202756293', '6303783743713708484', '6468365417517605478', '7887407208809957066', '8991483632660067955', '9384092744660032334', '10547779125865178270', '10595041987461739196', '11485414282913601829', '11760987759040078706', '11781010922933920259', '12926169240333359924', '13077487413648394209', '14241248046650668697', '14463102537742211332', '14579590163033179898', '14722230792170818214', '14988735547592816033', '15758769748652033371', '16084996898873488732', '16085621648536044385', '16836421281688546980', '16856013

# Document APIs

## Get document information without content

In [26]:
dataset = 'trump_tweets'
print('------------------- Get document information for {} -----------------------'.format(dataset))
# doc_titles, doc_ids, and metadata_selection below are filters to narrow down 
# documents to be retrieved.
# The information of all documents will be retrived when no filters are provided.

# doc_titles: list of strings
# The titles of the documents to retrieve. Example: ["title1", "title2", ..., "titleN"]  (optional)
# doc_titles = ['D_Trump2018_8_18_1_47']   
doc_titles = []
# doc_ids: list of strings
# The docid of the documents to retrieve. Example: ["docid1", "docid2", ..., "docidN"]  (optional)
# doc_ids = ['3397215194896514820', '776902852041351634']
doc_ids = []

# metadata_selection = {"author": "D_Trump16"} # dict | A selector off metadata. Example: {"field": "value"}  (optional)
metadata_selection = ''

try:
    payload = nucleus_api.DocInfo(dataset=dataset, 
                                doc_titles=doc_titles, 
                                doc_ids=doc_ids,
                                metadata_selection=metadata_selection)
    api_response = api_instance.post_doc_info(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

for res in api_response.result:
    print('Document ID:', res.sourceid)
    print('    title:', res.title)
    for attr in res.attribute.keys():
        if attr == 'time':
            print('   ', attr, ':', datetime.datetime.fromtimestamp(float(res.attribute[attr])))
        else:
            print('   ', attr, ':', res.attribute[attr])

    print('---------------')

print('-------------------------------------------------------------')

------------------- Get document information for trump_tweets -----------------------
Document ID: 372746459070796601
    title: D_Trump2018_8_14_1_57
    time : 2018-08-13 18:57:00
    author : D_Trump77
---------------
Document ID: 656244823936517128
    title: D_Trump2018_8_15_19_52
    time : 2018-08-15 12:52:00
    author : D_Trump48
---------------
Document ID: 776902852041351634
    title: D_Trump2018_8_13_16_9
    time : 2018-08-13 09:09:00
    author : D_Trump90
---------------
Document ID: 950604085993420810
    title: D_Trump2018_8_16_12_50
    time : 2018-08-16 05:50:00
    author : D_Trump40
---------------
Document ID: 1292265014981711161
    title: D_Trump2018_8_15_12_54
    time : 2018-08-15 05:54:00
    author : D_Trump62
---------------
Document ID: 1380411530707030282
    title: D_Trump2018_8_16_23_30
    time : 2018-08-16 16:30:00
    author : D_Trump34
---------------
Document ID: 1620156333107313580
    title: D_Trump2018_8_13_17_11
    time : 2018-08-13 10:11:00


## Display document info with a metadata selection

In [27]:
dataset = 'trump_tweets' # str | Dataset name.
print('------------------- Get document information for {} -----------------------'.format(dataset))

metadata_selection = {"author": "D_Trump16"}      # dict | A selector off metadata. Example: {"field": "value"}  (optional)

try:
    payload = nucleus_api.DocInfo(dataset=dataset, metadata_selection=metadata_selection)
    api_response = api_instance.post_doc_info(payload)
    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

for res in api_response.result:
    print('Document ID:', res.sourceid)
    print('    title:', res.title)
    for attr in res.attribute.keys():
        if attr == 'time':
            print('   ', attr, ':', datetime.datetime.fromtimestamp(float(res.attribute[attr])))
        else:
            print('   ', attr, ':', res.attribute[attr])

    print('---------------')

print('-------------------------------------------------------------')

------------------- Get document information for trump_tweets -----------------------
Document ID: 5864841412738683134
    title: D_Trump2018_8_17_19_25
    time : 2018-08-17 12:25:00
    author : D_Trump16
---------------
-------------------------------------------------------------


## Display document details

In [28]:
dataset = 'trump_tweets' # str | Dataset name.
print('------------------- Get document details for {} -----------------------'.format(dataset))

#doc_titles = ['D_Trump2018_8_18_1_47']   # str | The title of the documents to retrieve. Example: ["title1", "title2", ..., "titleN"]  (optional)
doc_ids = ['776902852041351634']      # str | The docid of the documents to retrieve. Example: ["docid1", "docid2", ..., "docidN"]  (optional)

try:
    payload = nucleus_api.DocDisplay(dataset, doc_ids=doc_ids)
    api_response = api_instance.post_doc_display(payload)
    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

for res in api_response.result:
    print('Document ID:', res.sourceid)
    print('    Title:', res.title)
    print('    Author:', res.attribute['author'])
    print('    Time:', datetime.datetime.fromtimestamp(float(res.attribute['time'])))
    print('    Content', res.content)

    print('---------------')

print('-------------------------------------------------------------')

------------------- Get document details for trump_tweets -----------------------
Document ID: 776902852041351634
    Title: D_Trump2018_8_13_16_9
    Author: D_Trump90
    Time: 2018-08-13 09:09:00
    Content None
---------------
-------------------------------------------------------------


## Display document details with a metadata selection

In [29]:
dataset = 'trump_tweets' # str | Dataset name.
print('------------------- Get document details for {} -----------------------'.format(dataset))
metadata_selection = {"author": "D_Trump16"}      # dict | A selector off metadata. Example: {"field": "value"}  (optional)

try:
    payload = nucleus_api.DocDisplay(dataset=dataset, metadata_selection=metadata_selection)
    api_response = api_instance.post_doc_display(payload)
    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

for res in api_response.result:
    print('Document ID:', res.sourceid)
    print('    Title:', res.title)
    print('    Author:', res.attribute['author'])
    print('    Time:', datetime.datetime.fromtimestamp(float(res.attribute['time'])))
    print('    Content', res.content)

    print('---------------')

print('-------------------------------------------------------------')

------------------- Get document details for trump_tweets -----------------------
Document ID: 5864841412738683134
    Title: D_Trump2018_8_17_19_25
    Author: D_Trump16
    Time: 2018-08-17 12:25:00
    Content None
---------------
-------------------------------------------------------------


## Get document recommendations

In [30]:
dataset = 'trump_tweets' # str | Dataset name.
print('------------- Get document recommendations for {} -----------------'.format(dataset))

#query = '("Trump" OR "president")' # str | Fulltext query, using mysql MATCH boolean query format. Example, (\"word1\" OR \"word2\") AND (\"word3\" OR \"word4\") (optional)
query = ''
custom_stop_words = ["real","hillary"] # str | List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)

try:
    payload = nucleus_api.DocumentRecommendModel(dataset=dataset, 
                                                query=query, 
                                                custom_stop_words=custom_stop_words, 
                                                num_topics=num_topics, 
                                                num_keywords=num_keywords)
    api_response = api_instance.post_doc_recommend_api(payload)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    
for i, res in enumerate(api_response.result):
    print('Document recommendations for topic', i, ':')
    print('    Keywords:', res.keywords)

    for j, doc in enumerate(res.recommendations):
        print('    Recommendation', j, ':')
        print('        Document ID:', doc.sourceid)
        print('        Title:', doc.title)
        print('        Attribute:', doc.attribute)
        print('        Author:', doc.attribute['author'])
        print('        Time:', datetime.datetime.fromtimestamp(float(doc.attribute['time'])))
    
    print('---------------')
    
print('-------------------------------------------------------------')

------------- Get document recommendations for trump_tweets -----------------
Document recommendations for topic 0 :
    Keywords: illegal aliens;united permission;permission illegal;enter united;aliens treated;aliens illegal;treated entered;entered legally
    Recommendation 0 :
        Document ID: 3397215194896514820
        Title: D_Trump2018_8_15_13_18
        Attribute: {'docid': 3397215194896514820, 'author': 'D_Trump57', 'time': 1534339080}
        Author: D_Trump57
        Time: 2018-08-15 06:18:00
    Recommendation 1 :
        Document ID: 4825367511331474696
        Title: D_Trump2018_8_15_12_44
        Attribute: {'docid': 4825367511331474696, 'author': 'D_Trump63', 'time': 1534337040}
        Author: D_Trump63
        Time: 2018-08-15 05:44:00
---------------
Document recommendations for topic 1 :
    Keywords: bruce ohr;ohr justice;justice department;christopher steele;helping disgraced;disgraced christopher;believe accused;accused helping
    Recommendation 0 :
        

## Get document summary

In [31]:
dataset = 'trump_tweets' # str | Dataset name.
doc_title = 'D_Trump2018_8_17_14_10' # str | The title of the document to be summarized.
print('------------------ Get document summary for {} in {}  --------------------'.format(doc_title, dataset))

custom_stop_words = ["real","hillary"] # List of stop words. (optional)
summary_length = 6 # int | The maximum number of bullet points a user wants to see in the document summary. (optional) (default to 6)
context_amount = 0 # int | The number of sentences surrounding key summary sentences in the documents that they come from. (optional) (default to 0)
short_sentence_length = 0 # int | The sentence length below which a sentence is excluded from summarization (optional) (default to 4)
long_sentence_length = 40 # int | The sentence length beyond which a sentence is excluded from summarization (optional) (default to 40)

try:
    payload = nucleus_api.DocumentSummaryModel(dataset=dataset, 
                                            doc_title=doc_title, 
                                            custom_stop_words=custom_stop_words, 
                                            summary_length=summary_length, 
                                            context_amount=context_amount,
                                            short_sentence_length=short_sentence_length,
                                            long_sentence_length=long_sentence_length)
    api_response = api_instance.post_doc_summary_api(payload)
    
    print('Summary for', api_response.result.doc_title)
    for sent in api_response.result.summary.sentences:
        print('    *', sent)
    
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

print('-------------------------------------------------------------')

------------------ Get document summary for D_Trump2018_8_17_14_10 in trump_tweets  --------------------
Summary for D_Trump2018_8_17_14_10
    * Big pushback on Governor Andrew Cuomo of New York for his really dumb statement about America’s lack of greatness.
-------------------------------------------------------------


# Summarize file from URL 

In [32]:
######################################################################################
# file_params fields descriptions:  
#   file_url              : string, the URL at which the file is stored (could be a S3 bucket address for instance)
#   filename              : OPTIONAL string, filename saved on the server. also serves as the doc_title for summarization
#   custom_stop_words     : OPTIONAL a string list, user-provided list of stopwords to be excluded from the content analysis leading to document summarization
#                            ["word1", "word2", ...]. DEFAULT: empty
#   summary_length        : OPTIONAL an integer, the maximum number of bullet points a user wants to see in the document summary. DEFAULT: 6
#   context_amount        : OPTIONAL an integer, the number of sentences surrounding key summary sentences in the original document that a user wants to see in the document summary. DEFAULT: 0
#   short_sentence_length : OPTIONAL an integer, the sentence length below which a sentence is excluded from summarization. DEFAULT: 4 words
#   long_sentence_length  : OPTIONAL an integer, the sentence length beyond which a sentence is excluded from summarization. DEFAULT: 40 words
#
file_params = {
    'file_url': 'https://s3-us-west-2.amazonaws.com/sumup-public/nucleus-sdk/quarles20181109a.docx',
    'filename': 'quarles20181109a-newname.pdf',   
    'custom_stop_words': ["document", "sometimes"], 
    'summary_length': 6,
    'context_amount': 0, 
    'short_sentence_length': 4, 
    'long_sentence_length': 40}

result = nucleus_helper.summarize_file_url(api_instance, file_params)
  
print('Summary for', result.doc_title, ':')
for sent in result.summary.sentences:
    print('    *', sent)

print('-------------------------------------------------------------')

Summary for quarles20181109a-newname.pdf :
    * But the stress test conducted by the Federal Reserve is only one part of our stress testing regime.
    * Stress Capital Buffer
Many of you are familiar with the Federal Reserve’s proposal to integrate the stress test with the regulatory capital rule--known as the stress capital buffer (SCB).1 I believe the SCB proposal represents an important milestone as we enter the next chapter of our stress testing regime.
    * Transparency
Transparency of the stress test and its inputs and outputs is key to the credibility of the stress test, and there are several initiatives underway to provide additional transparency regarding the supervisory stress test models and scenario design process.
    * We are currently considering options to provide additional transparency regarding scenarios and scenario design and I expect that the Board will seek comment on the advisability of, and possible approaches to, gathering the public’s input on scenarios an

## Get document sentiment

In [33]:
dataset = 'trump_tweets' # str | Dataset name.
doc_title = 'D_Trump2018_8_17_14_10' # str | The title of the document to be analyzed.
print('------------------ Get document sentiment  for {} in {}  --------------------'.format(doc_title, dataset))

custom_stop_words = ["real","hillary"] # List of stop words. (optional)
num_topics = 8 # int | Number of topics to be extracted from the document. (optional) (default to 8)
num_keywords = 8 # int | Number of keywords per topic that is extracted from the document. (optional) (default to 8)

try:
    payload = nucleus_api.DocumentSentimentModel(dataset=dataset, 
                                                doc_title=doc_title, 
                                                custom_stop_words=custom_stop_words, 
                                                num_topics=num_topics, 
                                                num_keywords=num_keywords)
    api_response = api_instance.post_doc_sentiment_api(payload)
    
    print('Sentiment for', api_response.result.doc_title)
    print(api_response.result.sentiment)

except ValueError as e:
    print('ERROR:', e)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

print('-------------------------------------------------------------')

------------------ Get document sentiment  for D_Trump2018_8_17_14_10 in trump_tweets  --------------------
Sentiment for D_Trump2018_8_17_14_10
-0.3333
-------------------------------------------------------------


## Tag documents

In [35]:
dataset = 'trump_tweets' # str | Dataset name.
print('---------------- Tag dataset ------------------------')

try:
    payload = nucleus_api.DatasetTagging(dataset=dataset, 
                                        query='new york city OR big apple', 
                                        metadata_selection='', 
                                        time_period='',
                                        period_start='2010-01-01',
                                        period_end='2019-04-30')
    api_response = api_instance.post_dataset_tagging(payload)
    print('    Entities tagged:', api_response.result.entities_tagged)
    print('    Docids tagged with the entities:', api_response.result.doc_ids)
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])

---------------- Tag dataset ------------------------
    Entities tagged: ['new york city ', ' big apple']
    Docids tagged with the entities: ['12945860735388879748', '8047817457772465264', '15030888264722461978', '10595041987461739196', '16836421281688546980']
